In [2]:
# Upgrade Plotly to latest (>=6.1.1)
!pip install -U plotly

# Upgrade Kaleido to latest (>=0.2.1)
!pip install -U kaleido


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Airfare Insights Dashboard.ipynb

import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Load cleaned data
df = pd.read_csv("after cleaning data/Indian_Airline_data.csv")
df["route"] = df["source_city"] + " → " + df["destination_city"]
df["price_per_minute"] = (df["price"] / df["Duration_in_Minutes"]).round(2)

# SQL Setup
conn = sqlite3.connect(":memory:")
df.to_sql("flights", conn, index=False, if_exists="replace")

# SQL Queries
sql_queries = {
    "avg_price_by_airline": """
        SELECT airline, ROUND(AVG(price), 2) AS avg_price
        FROM flights
        GROUP BY airline
        ORDER BY avg_price DESC;
    """,
    "stops_vs_avg_price": """
        SELECT total_stop AS stops, ROUND(AVG(price), 2) AS avg_price
        FROM flights
        GROUP BY stops
        ORDER BY stops;
    """,
    "avg_price_by_class_and_route": """
        SELECT class, route, ROUND(AVG(price), 2) AS avg_price
        FROM flights
        GROUP BY class, route
        ORDER BY avg_price DESC
        LIMIT 10;
    """,
    "departure_time_vs_avg_price": """
        SELECT departure_time, ROUND(AVG(price), 2) AS avg_price
        FROM flights
        GROUP BY departure_time
        ORDER BY avg_price DESC;
    """,
    "days_left_vs_avg_price": """
        SELECT CASE 
                   WHEN days_left BETWEEN 0 AND 3 THEN '0-3 Days'
                   WHEN days_left BETWEEN 4 AND 7 THEN '4-7 Days'
                   WHEN days_left BETWEEN 8 AND 15 THEN '8-15 Days'
                   WHEN days_left BETWEEN 16 AND 30 THEN '16-30 Days'
                   ELSE '30+ Days'
               END AS days_left_bucket,
               ROUND(AVG(price), 2) AS avg_price
        FROM flights
        GROUP BY days_left_bucket
        ORDER BY avg_price DESC;
    """
}

sql_outputs = {name: pd.read_sql(query, conn) for name, query in sql_queries.items()}

# Dashboard Output Directory
os.makedirs("dashboard", exist_ok=True)

# KPI Summary
total_flights = len(df)
avg_price = round(df["price"].mean(), 2)
top_airline = df["airline"].value_counts().idxmax()
top_airline_flights = df["airline"].value_counts().max()

fig_summary = go.Figure()
fig_summary.add_trace(go.Indicator(mode="number", value=total_flights, title="Total Flights", domain={"row": 0, "column": 0}))
fig_summary.add_trace(go.Indicator(mode="number", value=avg_price, title="Average Price (₹)", domain={"row": 0, "column": 1}))
fig_summary.add_trace(go.Indicator(mode="number+delta", value=top_airline_flights, delta={"reference": df["airline"].value_counts().mean()}, title=f"Top Airline: {top_airline}", domain={"row": 0, "column": 2}))
fig_summary.update_layout(grid={"rows": 1, "columns": 3, "pattern": "independent"}, title="Airfare Summary KPIs", height=250)
fig_summary.write_image("dashboard/summary_kpis.png")
fig_summary.show()

# Main Dashboard
fig_dashboard = make_subplots(rows=2, cols=2, subplot_titles=(
    "Avg Price by Airline", "Stops vs Price",
    "Departure Time vs Price", "Days Left vs Price"
))

# Add Subplot Traces
fig_dashboard.add_trace(go.Bar(x=sql_outputs['avg_price_by_airline']['avg_price'],
                               y=sql_outputs['avg_price_by_airline']['airline'], orientation='h'), row=1, col=1)

fig_dashboard.add_trace(go.Bar(x=sql_outputs['stops_vs_avg_price']['stops'],
                               y=sql_outputs['stops_vs_avg_price']['avg_price']), row=1, col=2)

fig_dashboard.add_trace(go.Bar(x=sql_outputs['departure_time_vs_avg_price']['departure_time'],
                               y=sql_outputs['departure_time_vs_avg_price']['avg_price']), row=2, col=1)

fig_dashboard.add_trace(go.Bar(x=sql_outputs['days_left_vs_avg_price']['days_left_bucket'],
                               y=sql_outputs['days_left_vs_avg_price']['avg_price']), row=2, col=2)

fig_dashboard.update_layout(
    height=800, width=1000,
    title_text="Flight Pricing Dashboard (SQL Analysis)",
    showlegend=False
)

fig_dashboard.write_image("dashboard/airfare_dashboard.png")
fig_dashboard.show()


In [5]:
import plotly.graph_objects as go

# Create figure and add all traces
fig_dropdown = go.Figure()

# Trace 1 - Avg Price by Airline
trace1 = go.Bar(
    x=sql_outputs['avg_price_by_airline']['avg_price'],
    y=sql_outputs['avg_price_by_airline']['airline'],
    orientation='h',
    name='Avg Price by Airline',
    visible=True
)
fig_dropdown.add_trace(trace1)

# Trace 2 - Stops vs Price
trace2 = go.Bar(
    x=sql_outputs['stops_vs_avg_price']['stops'],
    y=sql_outputs['stops_vs_avg_price']['avg_price'],
    name='Stops vs Avg Price',
    visible=False
)
fig_dropdown.add_trace(trace2)

# Trace 3 - Departure Time vs Price
trace3 = go.Bar(
    x=sql_outputs['departure_time_vs_avg_price']['departure_time'],
    y=sql_outputs['departure_time_vs_avg_price']['avg_price'],
    name='Departure Time vs Avg Price',
    visible=False
)
fig_dropdown.add_trace(trace3)

# Trace 4 - Days Left vs Price
trace4 = go.Bar(
    x=sql_outputs['days_left_vs_avg_price']['days_left_bucket'],
    y=sql_outputs['days_left_vs_avg_price']['avg_price'],
    name='Days Left vs Avg Price',
    visible=False
)
fig_dropdown.add_trace(trace4)

# Dropdown buttons with layout updates
dropdown_buttons = [
    dict(label='Avg Price by Airline',
         method='update',
         args=[
             {'visible': [True, False, False, False]},
             {'title': 'Avg Price by Airline',
              'xaxis': {'title': 'Average Price (₹)'},
              'yaxis': {'title': 'Airline'}}]),

    dict(label='Stops vs Price',
         method='update',
         args=[
             {'visible': [False, True, False, False]},
             {'title': 'Stops vs Average Price',
              'xaxis': {'title': 'Number of Stops'},
              'yaxis': {'title': 'Average Price (₹)'}}]),

    dict(label='Departure Time vs Price',
         method='update',
         args=[
             {'visible': [False, False, True, False]},
             {'title': 'Departure Time vs Average Price',
              'xaxis': {'title': 'Departure Time Slot'},
              'yaxis': {'title': 'Average Price (₹)'}}]),

    dict(label='Days Left vs Price',
         method='update',
         args=[
             {'visible': [False, False, False, True]},
             {'title': 'Days Left vs Average Price',
              'xaxis': {'title': 'Days Left Bucket'},
              'yaxis': {'title': 'Average Price (₹)'}}]),
]

# Final layout setup
fig_dropdown.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=dropdown_buttons,
        x=0.5,
        xanchor="center",
        y=1.15,
        yanchor="top"
    )],
    title="Avg Price by Airline",
    showlegend=False,
    height=650,
    width=950,
    xaxis=dict(title='Average Price (₹)'),
    yaxis=dict(title='Airline'),
    template='plotly_white'
)

# Optional: Improve trace visuals
fig_dropdown.update_traces(
    hoverinfo='x+y+name',
    marker=dict(color='lightskyblue', line=dict(color='black', width=0.8))
)

# Save & show
fig_dropdown.write_html("dashboard/interactive_airfare_dashboard.html")
fig_dropdown.show()
